In [94]:
import os
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
from api.data.excel_conversion import Team, Division, Game

In [95]:
season = 2023
name = f'{season}-{season+1}'
playoff_teams = 7
regular_season_week_count = 18
playoff_game_names = ['Wild Card', 'Divisional', 'Conference Championship', 'Super Bowl']
playoff_name_conversions = {
    'WildCard': regular_season_week_count + 1,
    'Division': regular_season_week_count + 2,
    'ConfChamp': regular_season_week_count + 3,
    'SuperBowl': regular_season_week_count + 4,
}

In [96]:
def read_season_from_excel(path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    games_df: pd.DataFrame = pd.read_excel(path, 'Games')
    games_df.drop(columns=['Day', 'Unnamed: 7', 'YdsW', 'TOW', 'YdsL', 'TOL'], axis=1, inplace=True)
    games_df.rename(columns={'Unnamed: 5': 'At'}, inplace=True)
    games_df = games_df[games_df['Week'].notna()]
    games_df['At'] = games_df['At'].astype(str).str.replace('nan','')
    
    teams_df: pd.DataFrame = pd.read_excel(path, 'Teams')
    
    return games_df, teams_df

In [97]:
# get connection string from environment variable
load_dotenv()
file_path: str = os.getenv('Games_2024')

games_df, teams_df = read_season_from_excel(file_path)

teams_df.head()

,Location,Name,Division
0,Buffalo,Bills,AFC East
1,Miami,Dolphins,AFC East
2,New York,Jets,AFC East
3,New England,Patriots,AFC East
4,Kansas City,Chiefs,AFC West


In [99]:
def create_game(row: pd.Series, regular_season_week_count: int, playoff_game_names: list[str], playoff_name_conversions: dict[str, int]) -> Game:
    """
    Creates a Game object from a pandas series representing a single NFL game.

    Parameters:
    row (pd.Series): A pandas series containing the game data.
    regular_season_week_count (int): The number of weeks in the regular season.
    playoff_game_names (list[str]): The names of the playoff games, in order.
    playoff_name_conversions (dict[str, int]): A dictionary mapping playoff game names to their index in the list of playoff games.

    Returns:
    Game: A Game object representing the given game data.

    """
    date: datetime.date = row['Date'].date()
    time: datetime.time = datetime.strptime(row['Time'], "%I:%M%p").time()
    start_time = datetime.combine(date, time)

    # regular season
    if isinstance(row['Week'], int):
        week: int = row['Week']
        week_name: str = f'Week {row['Week']}'
    # playoffs
    else:
        week: int = playoff_name_conversions[row['Week']]
        week_name: str = playoff_game_names[week - regular_season_week_count - 1]

    if row['At'] == '@':
        away_team = row['Winner/tie']
        home_team = row['Loser/tie']
        away_score = row['PtsW']
        home_score = row['PtsL']
    else:
        away_team = row['Loser/tie']
        home_team = row['Winner/tie']
        away_score = row['PtsL']
        home_score = row['PtsW']

    return Game(
        week=week,
        week_name=week_name,
        away_team=away_team,
        home_team=home_team,
        away_score=away_score,
        home_score=home_score,
        game_time=start_time
    )
        
def create_teams_and_divisions(row: pd.Series) -> pd.Series:
    """
    Creates a Team and Division object from a pandas series representing a NFL team.

    Parameters:
    row (pd.Series): A pandas series containing the team data.

    Returns:
    pd.Series: A pandas series containing the team and division data.

    """
    location: str = row['Location']
    name: str = row['Name']
    full_name: str = f'{location} {name}'
    division: str = row['Division']
    conference = division[:3]

    team = Team(location=location, name=name, full_name=full_name, division=division)
    division = Division(name=division, conference=conference)

    row['Team'] = team
    row['Division'] = division

    return row
    

games_df['Game'] = games_df.apply(create_game, regular_season_week_count=regular_season_week_count, playoff_game_names=playoff_game_names, playoff_name_conversions=playoff_name_conversions, axis=1)
teams_df = teams_df.apply(create_teams_and_divisions, axis=1)

,Week,Date,Time,Winner/tie,At,Loser/tie,PtsW,PtsL,Game
0,1,2023-09-07 00:00:00,8:20PM,Detroit Lions,@,Kansas City Chiefs,21.0,20.0,"Game(week=1, week_name='Week 1', game_time=dat..."
1,1,2023-09-10 00:00:00,1:00PM,Atlanta Falcons,,Carolina Panthers,24.0,10.0,"Game(week=1, week_name='Week 1', game_time=dat..."
2,1,2023-09-10 00:00:00,1:00PM,Cleveland Browns,,Cincinnati Bengals,24.0,3.0,"Game(week=1, week_name='Week 1', game_time=dat..."
3,1,2023-09-10 00:00:00,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,31.0,21.0,"Game(week=1, week_name='Week 1', game_time=dat..."
4,1,2023-09-10 00:00:00,1:00PM,Washington Commanders,,Arizona Cardinals,20.0,16.0,"Game(week=1, week_name='Week 1', game_time=dat..."


In [ ]:
games: list[Game] = games_df['Game'].to_list()
teams: list[Team] = teams_df['Team'].to_list()
divisions: list[Division] = teams_df['Division'].to_list()